In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dropout, Attention
from tensorflow.keras.models import Model
import re
import numpy as np
from tensorflow.keras.models import load_model

import pandas as pd
import glob

2023-11-18 20:28:05.859490: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-18 20:28:06.446471: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-18 20:28:06.449495: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-18 20:28:10.664703: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [18]:
import pandas as pd
import re
import glob

file_pattern = f"data/conve*.txt"
matching_files = glob.glob(file_pattern)

input_texts = []
output_texts = []
df = pd.DataFrame()
for file_path in matching_files:
    with open(file_path, "r") as file:
        content = file.read()

    examples = content.split("assistant: ")

    inputs = []
    outputs = []

    i = 1
    for example in examples[1:]:
        for _ in range(100):
            input_match = re.search(f"Input{i}: (.+?)\n", example)
            output_match = re.search(f"Output{i}: (.+?)\n", example)

            if input_match and output_match:
                # outputs_value = "flowchart TD \n\t" + output_match.group(1)
                outputs_value = output_match.group(1)
                # outputs_value = format_mermaid_code(outputs_value)
                inputs_value = re.sub(r'Output.*', r'', input_match.group(1))
                inputs.append(inputs_value)
                outputs.append(outputs_value)
                output_texts.append(outputs_value)
                input_texts.append(inputs_value)
            i += 1  
    newDf = pd.DataFrame({"input": inputs, "output": outputs})
    df = pd.concat([df, newDf], ignore_index=True)



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Concatenate
from tensorflow.keras.models import Model
import numpy as np

# Define the maximum sequence length for input and output sequences
max_input_seq_len = max(len(seq) for seq in input_texts)
max_output_seq_len = max(len(seq) for seq in output_texts)

# Define the input and output tokenizers
input_tokenizer = Tokenizer(filters='', char_level=False)
output_tokenizer = Tokenizer(filters='', char_level=False)

# Fit the input tokenizer on the preprocessed input sequences
input_tokenizer.fit_on_texts(input_texts)

# Fit the output tokenizer on the preprocessed output sequences
output_tokenizer.fit_on_texts(output_texts)

# Convert the input and output sequences to numerical format
encoder_input_seqs = input_tokenizer.texts_to_sequences(input_texts)
decoder_input_seqs = output_tokenizer.texts_to_sequences(output_texts)
decoder_target_seqs = [seq[1:] for seq in decoder_input_seqs]

# Pad the input and output sequences to fixed length
encoder_input_seqs = pad_sequences(encoder_input_seqs, maxlen=max_input_seq_len, padding='post')
decoder_input_seqs = pad_sequences(decoder_input_seqs, maxlen=max_output_seq_len, padding='post')
decoder_target_seqs = pad_sequences(decoder_target_seqs, maxlen=max_output_seq_len, padding='post')

# Define the model architecture
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(max_input_seq_len,))
encoder_embedding = Embedding(input_dim=len(input_tokenizer.word_index) + 1, output_dim=latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(latent_dim, return_sequences=True, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_output_seq_len,))
decoder_embedding = Embedding(input_dim=len(output_tokenizer.word_index) + 1, output_dim=latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Attention layer
attention = Attention()([decoder_outputs, encoder_lstm])
context = Concatenate(axis=-1)([attention, decoder_outputs])

# Dense layer for prediction
decoder_dense = Dense(len(output_tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(context)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([encoder_input_seqs, decoder_input_seqs], decoder_target_seqs, epochs=1, batch_size=1)

# Save the model
model.save('my_model3.h5')


In [14]:
# Define the maximum sequence length for the input and output sequences
max_input_seq_len = max(len(seq) for seq in input_texts)
max_output_seq_len = max(len(seq) for seq in output_texts)

# Define the input and output tokenizers
input_tokenizer = Tokenizer(filters='', char_level=False)
output_tokenizer = Tokenizer(filters='', char_level=False)

# Fit the input tokenizer on the preprocessed input sequences
input_tokenizer.fit_on_texts(input_texts)

# Fit the output tokenizer on the preprocessed output sequences
output_tokenizer.fit_on_texts(output_texts)

SOS_token = '<sos>'
EOS_token = '<eos>'

# Add the special tokens to the output tokenizer
output_tokenizer.word_index[SOS_token] = len(output_tokenizer.word_index) + 1
output_tokenizer.word_index[EOS_token] = len(output_tokenizer.word_index) + 1

output_tokenizer.index_word[len(output_tokenizer.index_word) + 1] = SOS_token
output_tokenizer.index_word[len(output_tokenizer.index_word) + 1] = EOS_token

In [15]:
# Define the number of encoder and decoder tokens
num_encoder_tokens = len(input_tokenizer.word_index) + 1
num_decoder_tokens = len(output_tokenizer.word_index) + 2

# Define the model architecture
latent_dim = 256

# Define encoder input layer
encoder_inputs = Input(shape=(max_input_seq_len,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim, mask_zero=True)(encoder_inputs)
encoder_dropout = Dropout(0.1)(encoder_embedding)
encoder_lstm1 = LSTM(latent_dim, return_sequences=True)(encoder_dropout)
encoder_lstm2 = LSTM(latent_dim, return_sequences=True)(encoder_lstm1)
encoder_lstm3 = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm3(encoder_lstm2)
encoder_states = [state_h, state_c]

# Define decoder input layers
decoder_inputs = Input(shape=(max_output_seq_len,))
decoder_embedding = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)(decoder_inputs)
decoder_dropout = Dropout(0.1)(decoder_embedding)
decoder_lstm1 = LSTM(latent_dim, return_sequences=True)(decoder_dropout, initial_state=encoder_states)
decoder_lstm2 = LSTM(latent_dim, return_sequences=True)(decoder_lstm1, initial_state=encoder_states)
decoder_lstm3 = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm3(decoder_lstm2, initial_state=encoder_states)

# Define attention layer
attention = Attention()([decoder_outputs, encoder_outputs])

# Define decoder output layer
decoder_dense = Dense(num_decoder_tokens - 1, activation='softmax')
decoder_outputs = decoder_dense(attention)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Convert the input and output sequences to numerical format
encoder_input_seqs = input_tokenizer.texts_to_sequences(input_texts)
decoder_input_seqs = [[output_tokenizer.word_index['<sos>']] + output_tokenizer.texts_to_sequences([seq])[0] for seq in output_texts]
decoder_target_seqs = [seq[1:] + [output_tokenizer.word_index['<eos>']] for seq in decoder_input_seqs]

# Pad the input and output sequences to fixed length
encoder_input_seqs = pad_sequences(encoder_input_seqs, maxlen=max_input_seq_len, padding='post')
decoder_input_seqs = pad_sequences(decoder_input_seqs, maxlen=max_output_seq_len, padding='post')
decoder_target_seqs = pad_sequences(decoder_target_seqs, maxlen=max_output_seq_len, padding='post')

In [18]:
batch_size = 32
num_batches = int(np.ceil(len(decoder_target_seqs) / batch_size))

for batch_num in range(num_batches):
    start_idx = batch_num * batch_size
    end_idx = min((batch_num + 1) * batch_size, len(decoder_target_seqs))

    batch_encoder_input_seqs = encoder_input_seqs[start_idx:end_idx]
    batch_decoder_input_seqs = decoder_input_seqs[start_idx:end_idx]
    batch_decoder_target_seqs = decoder_target_seqs[start_idx:end_idx]
    
    # Tworzenie macierzy one-hot dla aktualnej porcji danych
    current_batch_size = len(batch_decoder_target_seqs)
    decoder_target_seqs_onehot = np.zeros((current_batch_size, max_output_seq_len, len(output_tokenizer.word_index) + 1))

    for i, seq in enumerate(batch_decoder_target_seqs):
        for j, token in enumerate(seq):
            decoder_target_seqs_onehot[i, j, token] = 1.0

    # Trenowanie modelu na podstawie aktualnej porcji danych
    model.train_on_batch([batch_encoder_input_seqs, batch_decoder_input_seqs], decoder_target_seqs_onehot)
    print(f'Batch {batch_num+1}/{num_batches} trained')


2023-11-18 12:05:48.776339: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [32,653,7720]
	 [[{{node Placeholder/_2}}]]
2023-11-18 12:06:01.163949: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "110" frequency: 1800 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 6291456 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


Batch 1/72 trained


2023-11-18 12:06:15.505920: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [32,653,7720]
	 [[{{node Placeholder/_2}}]]


Batch 2/72 trained


2023-11-18 12:06:26.416952: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [32,653,7720]
	 [[{{node Placeholder/_2}}]]


...

Batch 70/72 trained


2023-11-18 12:26:05.329183: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [32,653,7720]
	 [[{{node Placeholder/_2}}]]


Batch 71/72 trained


2023-11-18 12:26:26.184509: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [21,653,7720]
	 [[{{node Placeholder/_2}}]]
2023-11-18 12:26:40.472259: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "110" frequency: 1800 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 6291456 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


Batch 72/72 trained


In [ ]:
# to jest niepotrzebne

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit([encoder_input_seqs, decoder_input_seqs], decoder_target_seqs_onehot,
          batch_size=64,
          epochs=800,
          validation_split=0.1)

In [ ]:
# to jest niepotrzebne

model.fit([encoder_input_seqs, decoder_input_seqs], decoder_target_seqs_onehot,
          batch_size=64,
          epochs=400,
          validation_split=0.1)

In [9]:
model = load_model('my_model2.h5')

In [16]:
# Load the trained model
model = load_model('my_model2.h5')

# Define the input sentence
input_sentence = "To make a cup of tea, start by grinding the beans. Afterwards, add the grounds to the tea maker and add water. Finally, press the brew button and enjoy your tea"

# Preprocess the input sentence
input_sequence = input_tokenizer.texts_to_sequences([input_sentence])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_input_seq_len, padding='post')

# Initialize the decoded sequence with beam search
beam_width = 3
decoded_sequences = np.zeros((beam_width, max_output_seq_len))

# Use beam search for decoding
for i in range(max_output_seq_len):
    predictions = model.predict([padded_input_sequence, decoded_sequences])
    
    # Select top-k indices using beam search
    top_k_indices = np.argsort(predictions[0, i, :])[-beam_width:][::-1]

    # Expand the beam
    new_sequences = decoded_sequences.copy()
    for j, idx in enumerate(top_k_indices):
        new_sequences[j, i] = idx

    decoded_sequences = new_sequences

# Convert the predicted sequences back to text
output_sentences = [output_tokenizer.sequences_to_texts([decoded_sequence]) for decoded_sequence in decoded_sequences]

print("Input sentence:", input_sentence)
print("Predicted outputs:")
for output_sentence in output_sentences:
    print(output_sentence)


ValueError: Data cardinality is ambiguous:
  x sizes: 1, 3
Make sure all arrays contain the same number of samples.

In [10]:
input_text = "To make a cup of tea, start by grinding the beans. Afterwards, add the grounds to the tea maker and add water. Finally, press the brew button and enjoy your tea"

# Tokenize the input sentence
input_seq = input_tokenizer.texts_to_sequences([input_text])[0]
# Pad the input sequence
input_seq = pad_sequences([input_seq], maxlen=max_input_seq_len, padding='post')

# Generate the output sequence using the trained model
decoder_input = np.zeros(shape=(len(input_seq), max_output_seq_len))
decoder_input[:, 0] = output_tokenizer.word_index['<sos>']
for i in range(1, max_output_seq_len):
    predictions = model.predict([input_seq, decoder_input]).argmax(axis=2)
    decoder_input[:, i] = predictions[:, i-1]

print(decoder_input)

KeyError: '<sos>'

In [ ]:
# Convert the output sequence to text
output_text = ''
for i in range(max_output_seq_len):
    # print(int(decoder_input[0,i]))
    # print(output_tokenizer.index_word)
    if output_tokenizer.index_word[int(decoder_input[0,i])] == '<sos>':
        continue
    if output_tokenizer.index_word[int(decoder_input[0,i])] == '<eos>':
        break
    else:
        output_text += output_tokenizer.index_word[int(decoder_input[0,i])] + ' '

In [22]:
# Print the input and output sentences
print('Input sentence:', input_text)
print('Output sentence:', output_text)

Input sentence: To make a cup of tea, start by grinding the beans. Afterwards, add the grounds to the tea maker and add water. Finally, press the brew button and enjoy your tea
Output sentence: --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --> --

In [ ]:
# Print the input and output sentences
print('Input sentence:', input_text)
print('Output sentence:', output_text)

In [ ]:
# Print the input and output sentences
print('Input sentence:', input_text)
print('Output sentence:', output_text)

#### irrelevant

In [15]:
# Preprocess the test examples
test_encoder_input_seqs = input_tokenizer.texts_to_sequences(input_texts[4:6])
test_encoder_input_seqs = pad_sequences(test_encoder_input_seqs, maxlen=max_input_seq_len, padding='post')

# Initialize the decoder input sequences with the SOS token
test_decoder_input_seqs = np.zeros((len(test_encoder_input_seqs), max_output_seq_len))
test_decoder_input_seqs[:, 0] = output_tokenizer.word_index[SOS_token]
print(test_decoder_input_seqs)

# Generate predictions on the test examples
predictions = model.predict([test_encoder_input_seqs, test_decoder_input_seqs])

# Convert the predictions to text format
predicted_texts = []
for prediction in predictions:
    predicted_seq = []
    for token_vec in prediction:
        token_index = np.argmax(token_vec)
        token = output_tokenizer.index_word[token_index]
        if token == EOS_token:
            break
        predicted_seq.append(token)
    predicted_texts.append(' '.join(predicted_seq))

# Print the predicted texts
for i, predicted_text in enumerate(predicted_texts):
    print("Test example", i+1, "predicted text:", predicted_text)

[[10137.     0.     0. ...     0.     0.     0.]
 [10137.     0.     0. ...     0.     0.     0.]]
1/1 [==============================] - 2s 2s/step
Test example 1 predicted text: --> e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[take e[ta